The below code will read raw data and chunk ready for loading into vector store and knowledge graph.


In [1]:
import os
import json


In [2]:
# Read the JSON file
with open("data/raw/index.json", "r") as json_file:
    raw_index = json.load(json_file)

# Set overwrite flag
overwrite = False

# llmsherpa


In [3]:
# from llmsherpa.readers import LayoutPDFReader

# # from llama_index.readers.schema.base import Document
# from llama_index.core.schema import Document
# from llama_index.core import VectorStoreIndex

In [4]:
# llmsherpa_api_url = "http://localhost:5010/api/parseDocument?renderFormat=all&applyOcr=yes"
# pdf_reader = LayoutPDFReader(llmsherpa_api_url)
# doc = pdf_reader.read_pdf(file)

In [5]:
# for chunk in doc.sections():
#     print(chunk.to_text(include_children=True, recurse=True))

# pymupdf4llm


In [6]:
import pymupdf4llm
from common import create_folder, clear_folder

img_dir = "data/processed/pymupdf4llm/images"
create_folder(img_dir)
if overwrite:
    clear_folder(img_dir)

path = "data/processed/pymupdf4llm/txt"
create_folder(path)
if overwrite:
    clear_folder(path)

Folder 'data/processed/pymupdf4llm/images' has been created (or already exists).
Folder 'data/processed/pymupdf4llm/txt' has been created (or already exists).


In [7]:

for file in raw_index["html_pages"] + raw_index["pdf_pages"]:

    output_file: str = os.path.join(path, os.path.splitext(os.path.basename(file["output_filename"]))[0] + ".txt")

    # Check if the file already exists and skip if overwrite is False
    if not overwrite and os.path.exists(output_file):
        print(f"Skipping {output_file} (already exists)")
        continue

    print(file["output_filename"])
    data: str = pymupdf4llm.to_markdown(file["output_filename"], write_images=True, page_chunks=True, image_path=img_dir)
    
    with open(output_file, "w") as f:
        for i, row in enumerate(data):
            f.write(f"Page: {row['metadata']['page']}\n")
            f.write(f"{row['text']}\n")
            f.write("\n")


data/raw/html/An Overview on RAG Evaluation.html
Processing data/raw/html/An Overview on RAG Evaluation.html...
[                                        ] (0/1===================

====================[========================================] (1/1]
data/raw/html/Evaluation Metrics for Search and Recommendation Systems.html
Processing data/raw/html/Evaluation Metrics for Search and Recommendation Systems.html...
[                                        ] (0/1=======================================[========================================] (1/1]
data/raw/html/Electric and hybrid cars.html
Processing data/raw/html/Electric and hybrid cars.html...
[                                        ] (0/1=======================================[========================================] (1/1]
data/raw/html/Is it worth buying a second-hand Nissan Leaf.html
Processing data/raw/html/Is it worth buying a second-hand Nissan Leaf.html...
[                                        ] (0/1=======================================[========================================] (1/1]
data/raw/pdf/Monthly operating report for February 2025 (PDF).pdf
Processing data/raw/pdf/Monthly operating report f

# PyMuPDF


In [8]:
import pymupdf

path = "data/processed/pymupdf/txt"
create_folder(path)

Folder 'data/processed/pymupdf/txt' has been created (or already exists).


In [9]:
if overwrite:
    clear_folder(path)

for file in raw_index["html_pages"] + raw_index["pdf_pages"]:

    output_file: str = os.path.join(path, os.path.splitext(os.path.basename(file["output_filename"]))[0] + ".txt")
    
    # Check if the file already exists and skip if overwrite is False
    if not overwrite and os.path.exists(output_file):
        print(f"Skipping {output_file} (already exists)")
        continue

    doc = pymupdf.open(file["output_filename"])
    i = 1
    with open(output_file, "w") as f:
        for row in doc:
            f.write(f"Page: {i}\n")
            f.write(row.get_text())
            f.write("\n")  # Add a new line after each entry
            i += 1